# Data Concatenator

## 필요한 Data
- surface level ERA5, precipitation 제외
- surface level ERA5, precipitation 포함
- pressure level ERA5
- (optional) TOA solar incident radiation

cmd에서 `.nc`로 변환하는 법
```
module load gcc/cdo-1.9.3
cdo -f nc copy input.grib output.nc
```

In [1]:
import xarray as xr
import numpy as np
import his_utils  # 사용자 정의 모듈
import dask
import cdsapi

import sys
import traceback

In [2]:

#==============================================================
#  Phase 1: Set Default Paths and Variables
#==============================================================
print("====================================================")
print("                   Phase 1: Setup                   ")
print("====================================================")

is_grib = True
default_path = "/geodata2/S2S/DL/GC_input"

if is_grib:
    ds_surface = f"{default_path}/suf.grib"
    ds_surface_precip = f"{default_path}/tp.grib"
    ds_pressure_level = f"{default_path}/pres.grib"
    ds_surface_precip_prior = None #f"{default_path}/tp_prior.grib"
else:
    ds_surface = "testdata/surface.nc"
    ds_surface_precip = "testdata/precip.nc"
    ds_surface_precip_prior = None
    ds_pressure_level = "testdata/pressure.nc"

# FIXME
year = 2021
month = 1
day = 4

year = [f"{year}"]
month = [f"{month}"]
day = int(day)
time = ["12:00", "18:00"]

if ds_surface_precip_prior is not None:
    day_prior = [f"{day-1}"]
    time_prior = [
        "12:00", "13:00", "14:00", "15:00", 
        "16:00", "17:00", "18:00", "19:00",
        "20:00", "21:00", "22:00", "23:00"
    ]

#==============================================================
#  Phase 2: Define CDS API Client and Requests
#==============================================================
print("====================================================")
print("      Phase 2: Define CDS API Client and Requests   ")
print("====================================================")

client = cdsapi.Client()
dataset = "reanalysis-era5-pressure-levels"
request = {
    "product_type": ["reanalysis"],
    "data_format": "grib",
    "download_format": "unarchived"
}

P_level_13 = [
    "50", "100", "150", "200", "250", 
    "300", "400", "500", "600","700", 
    "850", "925", "1000"
]

P_level_37 = [
    "1", "2", "3",
    "5", "7", "10",
    "20", "30", "50",
    "70", "100", "125",
    "150", "175", "200",
    "225", "250", "300",
    "350", "400", "450",
    "500", "550", "600",
    "650", "700", "750",
    "775", "800", "825",
    "850", "875", "900",
    "925", "950", "975",
    "1000"
    ]

dataset_single = "reanalysis-era5-single-levels"
dataset_multi = "reanalysis-era5-pressure-levels"

resolution = '0.25/0.25' # '1.0/1.0'

req_multi = {
    "product_type": ["reanalysis"],
    "variable": [
        "geopotential",
        "specific_humidity",
        "temperature",
        "u_component_of_wind",
        "v_component_of_wind",
        "vertical_velocity"
    ],
    "year": year,
    "month": month,
    "day": [f"{day}"],
    "time": time,
    "pressure_level": P_level_13,
    "data_format": "grib",
    "download_format": "unarchived",
    'grid': resolution
}

req_tp = {
    "product_type": ["reanalysis"],
    "variable": ["total_precipitation"],
    "year": year,
    "month": month,
    "day": [f"{day}"],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "grib",
    "download_format": "unarchived",
    'grid': resolution
}

req_surface = {
    "product_type": ["reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "2m_temperature",
        "mean_sea_level_pressure",
        "toa_incident_solar_radiation",
        "geopotential",
        "land_sea_mask"
    ],
    "year": year,
    "month": month,
    "day":[f"{day}"],
    "time": time,
    "data_format": "grib",
    "download_format": "unarchived",
    'grid': resolution
}


                   Phase 1: Setup                   
      Phase 2: Define CDS API Client and Requests   


2025-03-18 09:25:56,204 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-03-18 09:25:56,204 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


In [3]:

#==============================================================
#  Phase 3: Retrieve Data via CDS API
#==============================================================
print("====================================================")
print("              Phase 3: Retrieving Data              ")
print("====================================================")

# execute client.retrieve() concurrently by multiprocessing

from threading import Thread

def retrieve_data(client, dataset, request, output):
    client.retrieve(dataset, request, output)

t1 = Thread(target=retrieve_data, args=(client, dataset_multi, req_multi, ds_pressure_level))
t2 = Thread(target=retrieve_data, args=(client, dataset_single, req_tp, ds_surface_precip))
t3 = Thread(target=retrieve_data, args=(client, dataset_single, req_surface, ds_surface))

t1.start()
t2.start()  
t3.start()

t1.join()
t2.join()
t3.join()

if ds_surface_precip_prior is not None:
    print("Retrieving prior total_precipitation data...")
    req_tp_prior = {
        "product_type": ["reanalysis"],
        "variable": ["total_precipitation"],
        "year": year,
        "month": month,
        "day": day_prior,
        "time": [
            "00:00", "01:00", "02:00",
            "03:00", "04:00", "05:00",
            "06:00", "07:00", "08:00",
            "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00",
            "15:00", "16:00", "17:00",
            "18:00", "19:00", "20:00",
            "21:00", "22:00", "23:00"
        ],
        "data_format": "grib",
        "download_format": "unarchived",
        'grid': resolution
    }
    client.retrieve(    
        dataset_single,
        req_tp_prior,
        ds_surface_precip_prior
    )


              Phase 3: Retrieving Data              


2025-03-18 09:26:07,020 INFO Request ID is e324494c-79ad-4905-a5b3-11494ff60fce
2025-03-18 09:26:07,163 INFO Request ID is 9edbc12c-6f3b-4e7a-b4f2-39fa95d33fbf
2025-03-18 09:26:07,332 INFO status has been updated to accepted
2025-03-18 09:26:07,473 INFO status has been updated to accepted
2025-03-18 09:26:07,499 INFO Request ID is 4143a7b6-fcec-4725-a644-15695f876f52
2025-03-18 09:26:07,806 INFO status has been updated to accepted
2025-03-18 09:26:20,898 INFO status has been updated to running
2025-03-18 09:26:28,795 INFO status has been updated to accepted
2025-03-18 09:26:40,366 INFO status has been updated to successful
2025-03-18 09:26:40,475 INFO status has been updated to successful
2025-03-18 09:26:40,770 INFO status has been updated to successful


334e2b852382f38dec2bf3422cd46a07.grib:   0%|          | 0.00/47.5M [00:00<?, ?B/s]

3b7a53f19b7a661b7ef65f0d2df47e76.grib:   0%|          | 0.00/309M [00:00<?, ?B/s]

6c62d502f07f82d58b21a27cfb32fa69.grib:   0%|          | 0.00/31.7M [00:00<?, ?B/s]

In [1]:
import xarray as xr

xr.open_dataset("/geodata2/S2S/DL/GC_input/suf.grib", engine="cfgrib")

skipping variable: paramId==212 shortName='tisr'
Traceback (most recent call last):
  File "/home/hiskim1/.conda/envs/hiskim1_gencast/lib/python3.11/site-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/hiskim1/.conda/envs/hiskim1_gencast/lib/python3.11/site-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1609761600, 1609783200])) new_value=Variable(dimensions=(), data=1609740000)


<xarray.Dataset> Size: 50MB
Dimensions:     (time: 2, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 16B 2021-01-04T12:00:00 2021-01-04T18:0...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  (time) datetime64[ns] 16B ...
Data variables:
    u10         (time, latitude, longitude) float32 8MB ...
    v10         (time, latitude, longitude) float32 8MB ...
    t2m         (time, latitude, longitude) float32 8MB ...
    msl         (time, latitude, longitude) float32 8MB ...
    z           (time, latitude, longitude) float32 8MB ...
    lsm         (time, latitude, longitude) float32 8MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-03-18T09:42 GRIB to CDM+CF via cfgrib-0.9.1...

# 1. Surface Data w/o Precipitation

precipitation 제외하고 받은 6-hrly data 전처리 하는 과정

## TODO
- dimension 이름 변경
- 불필요한 coordinate 삭제
- variable 이름 변경 -> 나중에 합치고 한 번에 진행 가능

In [4]:

#==============================================================
#  Phase 4: Open and Pre-process Datasets
#==============================================================
print("====================================================")
print("          Phase 4: Open and Pre-process Data        ")
print("====================================================")

print("Opening ds_surface...")
if is_grib:
    ds1 = xr.open_dataset(ds_surface, engine='cfgrib')
    # print(ds1.data_vars)
    # ds1 = ds1.drop_vars(['number', 'step', 'surface', 'valid_time'])
    # ds1 = ds1.rename({"z" : "geopotential_at_surface"})
else:
    ds1 = xr.open_dataset(ds_surface)
    ds1 = ds1.rename({'var165':"u10",
                      'var166':"v10", 
                      'var167':"t2m", 
                      'var129':"geopotential_at_surface", 
                      'var172':"lsm", 
                      'var151':"msl", 
                      'var212':"tisr"})

          Phase 4: Open and Pre-process Data        
Opening ds_surface...


skipping variable: paramId==212 shortName='tisr'
Traceback (most recent call last):
  File "/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/cfgrib/dataset.py", line 721, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/cfgrib/dataset.py", line 639, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1609761600, 1609783200])) new_value=Variable(dimensions=(), data=1609740000)


# 2. Precipitation Data

Accumulates 6-hour precipitation into the next time step?
or into previous step?

## TODO
- coordinate 정리
- 강수량 합치기

In [ ]:

print("Opening and pre-processing ds_surface_precip (tp)...")
def sync_tp_coords(dataset: xr.Dataset):
    dataset = dataset.stack(new_time=['time', 'step'])
    dataset = dataset.assign_coords(new_time=dataset.valid_time.values)
    dataset = dataset.rename({'new_time': 'time'})
    dataset = dataset.drop_vars(['number', 'surface'])
    return dataset

if is_grib:
    ds2 = xr.open_dataset(ds_surface_precip, engine='cfgrib')
    if ds_surface_precip_prior is not None:
        prior = xr.open_dataset(ds_surface_precip_prior, engine='cfgrib')
        print("Syncing prior precipitation coords...")
        prior = sync_tp_coords(prior)

    print("Syncing ds2 coords...")
    ds2 = sync_tp_coords(ds2)

else:
    ds2 = xr.open_dataset(ds_surface_precip)
    ds2 = ds2.rename({'var228': 'tp'})
    if ds_surface_precip_prior is not None:
        prior = xr.open_dataset(ds_surface_precip_prior)
        prior = prior.rename({'var228': 'tp'})
ds2

Opening and pre-processing ds_surface_precip (tp)...
Syncing ds2 coords...
Slicing ds2 to remove first 5 and last 7 timesteps...


/tmp/ipykernel_1831064/1684770744.py:4: FutureWarning: updating coordinate 'new_time' with a PandasMultiIndex would leave the multi-index level coordinates ['time', 'step'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['new_time', 'time', 'step'])` before assigning new coordinate values.
  dataset = dataset.assign_coords(new_time=dataset.valid_time.values)


<xarray.Dataset>
Dimensions:     (latitude: 721, longitude: 1440, time: 36)
Coordinates:
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  (time) datetime64[ns] 2021-01-02T19:00:00 ... 2021-01-04T06:0...
  * time        (time) datetime64[ns] 2021-01-02T19:00:00 ... 2021-01-04T06:0...
Data variables:
    tp          (latitude, longitude, time) float32 nan nan nan ... nan nan nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-13T15:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [23]:
# 2-2. the time before
if is_grib and ds_surface_precip_prior is not None:
    print("Slicing prior to keep indices [5:29]...")
    prior = prior.isel(time=slice(5, 29))


In [24]:
# 2-3. merge the two datasets
if ds_surface_precip_prior is not None:
    print("Concatenating prior and ds2 along time dimension...")
    ds2 = xr.concat([prior, ds2], dim='time')

print("Sorting ds2 by time...")
ds2 = ds2.sortby('time')

print("Resampling ds2 in 6h intervals and summing precipitation...")
ds2 = ds2.resample(time='6h', closed='right', label='right').sum()

year = 2021
month = 1
day = 3

year = [f"{year}"]
month = [f"{month}"]
day = int(day)
time = ["12:00", "18:00"]

t_0 = f"{year[0]}-{month[0]:0>2}-{day:0>2}T{time[0]}:00"
t_1 = f"{year[0]}-{month[0]:0>2}-{day:0>2}T{time[1]}:00"

ds2 = ds2.sel(time=slice(t_0, t_1))

ds2

Sorting ds2 by time...
Resampling ds2 in 6h intervals and summing precipitation...


<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 2)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2021-01-03T12:00:00 2021-01-03T18:00:00
Data variables:
    tp         (time, latitude, longitude) float32 6.914e-05 6.914e-05 ... 0.0
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-13T15:04 GRIB to CDM+CF via cfgrib-0.9.1...

# 3. Pressure Level Data

37 level data를 처리하는 과정. 17 level이어도 동일한 방식

## TODO
- coordinate 정리

In [25]:

print("Opening ds_pressure_level (ds3)...")
if is_grib:
    ds3 = xr.open_dataset(ds_pressure_level, engine='cfgrib')
    ds3 = ds3.drop_vars(['number', 'step', 'valid_time'])
    ds3 = ds3.rename({"isobaricInhPa" : "level"})
    ds3 = ds3.sortby('level', ascending=True)
else:
    ds3 = xr.open_dataset(ds_pressure_level)
    ds3 = ds3.rename({'var130':"t", 
                      'var131':"u", 
                      'var132':"v", 
                      'var129':"z", 
                      'var133':"q", 
                      'var135':"w",
                      'plev': "level"})
    ds3['level'] = ds3['level']/100
    ds3['level'].attrs['units'] = 'hPa'
    ds3['level'].attrs['long_name'] = 'pressure level'


Opening ds_pressure_level (ds3)...


In [26]:

level = ds3.level.values
level = level.astype(np.int32)
ds3 = ds3.assign_coords(level=('level', level))


# (optional) 4. TOA 가공하기

구글에서 만든 거랑 내가 다운받은거랑 같다면 상관 없음.
다를 경우에는 이거 사용해야 함.

$\therefore$ $\exists$ noise
$\Rightarrow$ 결과가 미묘하게 달라지지만 유의미해보이지는 않다

In [27]:
# 4th. TOA
# ds4 = xr.open_dataset(ds_TOA, engine='cfgrib')

# ds4 = ds4.stack(new_time=['time', 'step'])
# ds4 = ds4.assign_coords(new_time=ds4.valid_time.values)
# ds4 = ds4.rename({'new_time': 'time'})
# ds4 = ds4.drop_vars(['number', 'surface', 'valid_time'])

# 4. 3개의 데이터셋을 하나로 합성하기

*ds1*, *ds2*, *ds3*, (*ds4*)를 하나로 합치고 GC에 잘 들어가도록 다듬어주기

## TODO
- 한 장씩 합치는 게 좋을지, 여러 장 한 번에 합치는 게 좋을지 for memory efficiency
- 합쳐서 GC에 잘 들어가는 지까지 확인

In [28]:
# merge all datasets
# ds_list = [ds1, ds2, ds3, ds4]
ds_list = [ds1, ds2, ds3]

result = xr.merge(ds_list)

for ds in ds_list:
    ds.close()

result = his_utils.transform_dataset(result)

## 4-1. 파일 명 정하기

In [29]:
result_path = 'testdata/2021-06-21/ERA5_input.nc'  #                     <------- FIX HERE

# 5. Output: GC로 준비 갈 완료!

In [30]:
result# .to_netcdf(result_path)

<xarray.Dataset>
Dimensions:                  (batch: 1, time: 2, lat: 721, lon: 1440, level: 37)
Coordinates:
  * batch                    (batch) int64 0
  * time                     (time) timedelta64[ns] 00:00:00 06:00:00
  * lat                      (lat) float64 -90.0 -89.75 -89.5 ... 89.75 90.0
  * lon                      (lon) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * level                    (level) int32 1 2 3 5 7 10 ... 900 925 950 975 1000
    datetime                 (batch, time) datetime64[ns] 2021-01-03T12:00:00...
Data variables: (12/13)
    10m_u_component_of_wind  (batch, time, lat, lon) float32 0.3667 ... 0.05989
    10m_v_component_of_wind  (batch, time, lat, lon) float32 -0.08539 ... 0.2161
    2m_temperature           (batch, time, lat, lon) float32 241.3 ... 255.7
    mean_sea_level_pressure  (batch, time, lat, lon) float32 9.963e+04 ... 1....
    geopotential_at_surface  (lat, lon) float32 2.736e+04 2.736e+04 ... 0.3228
    land_sea_mask            (lat, lon) float32 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    ...                       ...
    geopotential             (batch, time, level, lat, lon) float32 4.938e+05...
    specific_humidity        (batch, time, level, lat, lon) float32 3.936e-06...
    temperature              (batch, time, level, lat, lon) float32 287.8 ......
    u_component_of_wind      (batch, time, level, lat, lon) float32 0.002853 ...
    v_component_of_wind      (batch, time, level, lat, lon) float32 -0.001846...
    vertical_velocity        (batch, time, level, lat, lon) float32 -0.000256...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-13T14:56 GRIB to CDM+CF via cfgrib-0.9.1...

---
---
---
---
---
---
---
---
---
---
---
---

# 번외: xarray.resample은 어떻게 작동하는가?

| 1 | 2 | 3 | 4 | 5 | 6 | 7 | value

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | hr

`closed=` 어느 쪽을 닫힌 구간으로 쓸 것인가 $\Rightarrow$ 어느 쪽을 포함하고 반대쪽을 제외할까

`label=` sample한 거를 어느 쪽에 할당할 것인가

- case 1) `xarray.resample("6h")`

    1 + ... + 6을 0hr에 할당

- case 2) `xarray.resample("6h", closed='right', label='right')`

    2 + ... + 7을 6hr에 할당

In [ ]:
import pandas as pd

# 샘플 데이터 생성 (0시부터 23시까지)
date_range = pd.date_range(start='2021-12-31T19:00:00.000000000', end='2022-01-05T06:00:00.000000000', freq='h')
data = np.arange(1, 109, 1)
ds = pd.Series(data, index=date_range)

# 6시간 간격으로 리샘플링
ds_resampled = ds.resample('6h', closed='right', label='right').sum()

print("\n리샘플링 결과:")
print(ds_resampled)

# 각 리샘플링 구간의 시작과 끝 확인
for i, value in ds_resampled.items():
    start = i
    end = i + pd.Timedelta(hours=5)
    original_data = ds[start:end]
    print(f"\n{i}의 리샘플링 구간:")
    print(f"시작: {start}, 끝: {end}")
    print("포함된 원본 데이터:")
    print(original_data)
    print(f"합계: {value}")